In [29]:
class A(object):
    def foo(self):
        print("foo in A")

class B(A):
    def foo(self):
        print("foo in B")
        A.foo(self)


class C(B):
    def foo(self):
        print("foo in C")
        B.foo(self)
        
class D(C):
    def foo(self):
        print("foo in D")
        B.foo(self)
        C.foo(self)


In [30]:
kz = D()

In [31]:
kz.foo()

foo in D
foo in B
foo in A
foo in C
foo in B
foo in A


# To avoid multiple runs to each class:

In [38]:
class A(object):
    def foo(self):
        print("foo in A")

class B(A):
    def foo(self):
        print("foo in B")
        super(B,self).foo()
        #A.foo(self)


class C(B):
    def foo(self):
        print("foo in C")
        super(C,self).foo()
        #B.foo(self)
        
class D(C):
    def foo(self):
        print("foo in D")
        super(D,self).foo()
        #B.foo(self)
        #C.foo(self)


In [39]:
kz = D()

In [40]:
kz.foo()

foo in D
foo in C
foo in B
foo in A


In [1]:
class A(object):
    def foo(self):
        pass

class B(A):
    pass

class C(B):
    pass

In [2]:
OldDoc = None

In [3]:
def update_docstring(name, olddoc):
    # make sure it has a docstring
    if olddoc is None:
        return None

    # new docstring
    prefix = "%s: " % name
    if len(olddoc.split(": ")) > 1:
        newdoc = prefix + olddoc.split(": ")[1]
    else:
        newdoc = prefix + olddoc

    return newdoc

In [4]:
class RewriteDocstringMeta(type):
    """Modify docstrings to be prefixed with 'classname: '.
    To do this, we intercede before the class is created and modify the
    docstrings of its attributes.
    This will not affect inherited methods, however, so we also need to
    loop through the parent classes. We cannot simply modify the
    docstrings, because then the parent classes' methods will have the
    wrong docstring. Instead, we must actually copy the functions, and
    then modify the docstring.
    """

    def __new__(cls, name, parents, attrs):
        
        # loop through each attribute in attrs.
        for attr_name in attrs:
            # skip special methods
            if attr_name.startswith("__"): # special methods start with __
                continue

            # skip non-functions. 
            #__call__ is a feature of function attributes (bc they are callable)
            attr = attrs[attr_name]
            if not hasattr(attr, '__call__'): # if doesn't have __call__, not a function.
                continue

            # update docstring
            attr.__doc__ = update_docstring(name, attr.__doc__)

        for parent in parents:
            for attr_name in dir(parent):

                # we already have this method
                if attr_name in attrs:
                    continue

                # skip special methods
                if attr_name.startswith("__"):
                    continue

                # get the original function and copy it
                a = getattr(parent, attr_name)

                # skip non-functions
                if not hasattr(a, '__call__'):
                    continue

                # copy function
                f = a.__func__
                attr = type(f)(
                    f.func_code, f.func_globals, f.func_name,
                    f.func_defaults, f.func_closure)
                doc = f.__doc__

                # update docstring and add attr
                attr.__doc__ = update_docstring(name, doc)
                attrs[attr_name] = attr

        # create the class
        # super(classname, self).attributeName()
        obj = super(RewriteDocstringMeta, cls).__new__(
            cls, name, parents, attrs)
        return obj

In [5]:
class A(object):
    __metaclass__ = RewriteDocstringMeta

    def foo(self):
        """Do some stuff."""
        pass

class B(A):
    __metaclass__ = RewriteDocstringMeta

class C(B):
    __metaclass__ = RewriteDocstringMeta

In [6]:
print A.foo.__doc__ # prints 'A: Do some stuff.'
print B.foo.__doc__ # prints 'B: Do some stuff.'
print C.foo.__doc__ # prints 'C: Do some stuff.'

A: Do some stuff.
B: Do some stuff.
C: Do some stuff.


In [12]:
class PetOwner(object):
    '''
    class PetOwner:  holds personal information about the Pet Owner.
    '''
    __metaclass__ = RewriteDocstringMeta
    
    def __init__(self, name, age):
        self.name = name
        self.age = age
    
    def get_ownername(self):
        """Gets name"""
        return self.name

    def get_ownerage(self):
        """Gets age"""
        return self.age


class Pet(PetOwner):
    '''
    class Pet:  Holds information about the pet and 
    has access to the pet owner's attributes through class PetOwner
    '''
    __metaclass__ = RewriteDocstringMeta

    
    
    def __init__(self, name, age, petname, species, petage):
        self.petName = petname
        PetOwner.__init__(self, name, age)
        
    def get_petname(self):
        '''Get pets name'''
        return self.petName



In [17]:
print PetOwner.get_ownername.__doc__ # prints 'A: Do some stuff.'
print Pet.get_ownername.__doc__ # prints 'B: Do some stuff.'


PetOwner: Gets name
Pet: Gets name


In [18]:
patient1 = Pet('Kim Zoldak', 29, 'Sirius', 'dog', 6)

In [19]:
patient1.petName

'Sirius'

In [20]:
patient1.get_petname()

'Sirius'

In [24]:
patient1.age

29